In [1]:
import random
import numpy as np

# Parameters
num_nests = 20
max_iterations = 50
pa = 0.25  # Fraction of nests to abandon
num_phases = 3  # Number of traffic light phases per intersection

# Generate random traffic signal timings (e.g., seconds for each phase)
def random_timing():
    # Each phase between 10 and 60 seconds
    return np.random.uniform(10, 60, size=num_phases)

# Mock fitness function: lower total waiting time is better
def fitness(timing):
    # For demo, let's say waiting time increases with deviation from ideal timing (30 sec each phase)
    ideal = np.array([30, 30, 30])
    waiting_time = np.sum(np.abs(timing - ideal))
    # Add some randomness to simulate varying traffic conditions
    noise = random.uniform(-5, 5)
    return waiting_time + noise

# Levy flight step for new solution generation
def levy_flight(timing, alpha=0.1):
    # Levy flight: here simplified as random step scaled by alpha
    step = np.random.normal(0, 1, size=timing.shape)
    new_timing = timing + alpha * step * timing
    # Keep timings within valid bounds (10 to 60 seconds)
    new_timing = np.clip(new_timing, 10, 60)
    return new_timing

# Initialize nests
nests = [random_timing() for _ in range(num_nests)]

for iteration in range(max_iterations):
    for i in range(num_nests):
        new_timing = levy_flight(nests[i])
        new_fit = fitness(new_timing)

        j = random.randint(0, num_nests - 1)
        old_fit = fitness(nests[j])

        if new_fit < old_fit:
            nests[j] = new_timing

    # Abandon some nests and replace with new random timings
    num_abandon = int(pa * num_nests)
    fitness_values = [fitness(n) for n in nests]
    worst_nests_indices = sorted(range(num_nests), key=lambda k: fitness_values[k], reverse=True)[:num_abandon]

    for idx in worst_nests_indices:
        nests[idx] = random_timing()

    # Print best fitness every 10 iterations
    if iteration % 10 == 0 or iteration == max_iterations - 1:
        best_nest = min(nests, key=fitness)
        print(f"Iteration {iteration}: Best timing {best_nest}, Fitness: {fitness(best_nest):.2f}")

# Final best solution
best_nest = min(nests, key=fitness)
print("\nBest traffic signal timings found:", best_nest)
print("With estimated waiting time:", fitness(best_nest))

Iteration 0: Best timing [34.36564147 32.23394622 35.20078371], Fitness: 7.26
Iteration 10: Best timing [30.96896446 30.30768515 28.9600287 ], Fitness: 0.08
Iteration 20: Best timing [29.47627836 30.13353196 29.24382726], Fitness: -0.82
Iteration 30: Best timing [27.66648737 30.30826199 30.1986189 ], Fitness: 7.50
Iteration 40: Best timing [29.55918855 29.5374111  29.68193506], Fitness: 3.84
Iteration 49: Best timing [29.99724355 31.44266329 29.48930806], Fitness: 6.45

Best traffic signal timings found: [31.02621867 30.13052934 28.89048991]
With estimated waiting time: 2.660040348329776
